In [1]:
from market_growth_analysis.web_scraping import scrape_company_data, process_sector, process_prices_company, get_sector_data, get_industry_data, parser
import concurrent
import tqdm

from bs4 import BeautifulSoup
import requests

import yfinance as yf
import pandas as pd
import os
from os.path import join
from datetime import datetime
from pathlib import Path 

# Extract Financial Data


In [ ]:
URL = "https://www.macrotrends.net/stocks/research"
page = requests.get(URL)

sheets = ("income-statement",
          "cash-flow-statement",
          "balance-sheet",
          "financial-ratios")


soup = BeautifulSoup(page.content, "html.parser")

results = soup.find_all("a", href= lambda text: "/stocks/sector/" in text.lower())
sector_dict = dict()

for result in results:
    sector_dict[result.text] = ("".join(("https://www.macrotrends.net", result["href"]))).strip()

for sheet in sheets:
    with concurrent.futures.ThreadPoolExecutor() as executor:
        print("Retrieving income statement")
        financial_sheet = sheet
        financial_sheet_df = pd.DataFrame()

        for sector in tqdm.tqdm(sector_dict, desc="Sectors"):
            results = process_sector(sector, sector_dict, financial_sheet, executor)
            financial_sheet_df = pd.concat([financial_sheet_df, results], ignore_index=True)

        financial_sheet_df.to_csv(f'../../data/raw_01/{financial_sheet}.csv')
    


In [ ]:
financial_sheet_df

# Extract Prices

In [8]:
path = '../../data/raw_01/income-statement.csv'
companys = pd.read_csv(path).sort_values(by='ticker')['ticker'].dropna().unique()

In [9]:
companys

array(['A', 'AA', 'AAC', ..., 'ZYME', 'ZYNE', 'ZYXI'], dtype=object)

In [10]:
import concurrent.futures
from tqdm import tqdm
import logging

# Create a ThreadPoolExecutor
executor = concurrent.futures.ThreadPoolExecutor()

# List to store the future objects
futures = []

prices = pd.DataFrame()

# Iterate over the companies and submit the tasks to the executor
for company in companys:
    future = executor.submit(process_prices_company, company)
    futures.append(future)

# Use tqdm to track the progress of the loop
results = []
with tqdm(total=len(companys)) as pbar:
    for future in concurrent.futures.as_completed(futures):
        result = future.result()
        if result is not None:
            results.append(result)
        pbar.update(1)

# Combine the results
prices = pd.concat(results, ignore_index=True)


  0%|          | 8/5554 [00:01<10:31,  8.78it/s]  

- AAQC: No data found, symbol may be delisted


  1%|          | 53/5554 [00:05<10:19,  8.88it/s]

- ACH: No data found, symbol may be delisted


  7%|▋         | 380/5554 [00:35<09:16,  9.30it/s]

...

 97%|█████████▋| 5409/5554 [08:14<00:12, 11.86it/s]

- WPGGQ: No data found, symbol may be delisted


100%|█████████▉| 5538/5554 [08:26<00:01, 11.82it/s]

- ZME: No data found, symbol may be delisted
- ZNH: No data found, symbol may be delisted


100%|██████████| 5554/5554 [08:27<00:00, 10.95it/s]


In [11]:
# Export the dataframe
filepath = Path(f'../../data/raw_01/prices.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True)  
prices.to_csv(filepath)